# Tutorial 2 Inside the "optimize"

In [1]:
#!/usr/bin/env python3

# Standard python library

# Local library:
import IO.user_provided
import objective.setup_objective
import optimizer.gradient_free
import objective.compute_objective

# Third party library:

----------------------------------------------------------------------------------
###                       Taking the input from user                             
----------------------------------------------------------------------------------
#### Purposes: 
* Take the input from the command line

#### Return:
* main_logger: an object that defines the log file output ( you don't have to do anything with it )
* TOTAL_CORES: Number of cores assigned by slurm scheduler
* INPUT: a string of given input file name
* JOBID: a combination of Slurm job id and user-provided id
* Ref: Reference data address
* prep: template folder address for running sampling 

In [2]:
# For the interactive job: (Tutorial only):
main_logger,TOTAL_CORES,INPUT,JOBID,Ref,prep= (IO
                                         .user_provided
                                         .from_command_line(
                                            jobID="tutorial",
                                            total_cores=2,
                                            input_file="in_obj",
                                            ref_address="../force_matching_tutorial/ReferenceData",
                                            prep_address="../force_matching_tutorial/prepsystem")
                                         .finish_reading())


# Running the program from command (uncomment for the normal use): 
"""
main_logger, TOTAL_CORES, INPUT, JOBID,Ref,prep = (IO
                                          .user_provided
                                          .from_command_line()
                                          .finish_reading())
""" 
print ("cores requested:", TOTAL_CORES,type(TOTAL_CORES))
print ("input file name:", INPUT,type(INPUT))
print ("Job id:", JOBID,type(JOBID))
print ("Reference data path:", Ref,type(Ref))
print ("prepsystem data path:", prep,type(prep))

cores requested: 2 <class 'int'>
input file name: in_obj <class 'str'>
Job id: tutorial <class 'str'>
Reference data path: ../force_matching_tutorial/ReferenceData <class 'str'>
prepsystem data path: ../force_matching_tutorial/prepsystem <class 'str'>


----------------------------------------------------------------------------------
###                           Set up the workflow                                
----------------------------------------------------------------------------------
#### Purposes: 
* Set up working folders
* Initialize sampling methods such LAMMPS
* Parse the arugment of objective functions from the input file

#### Return: 
* ref_dict: a python dictionary containing reference data address. Each key is the name of matching type(force,rdf ...). Each value is another dictionary containing the arugment for the sub folder in that matching type. 
* predict_dict: a python dictionary containg predicted data address. Same data structure as ref_dict
* argument_dict: a python dictionary containing arguments needed to run objective functions inclduing number of cores requested, buffersize etc ... 
* LAMMPS: a Python class instance with the method: "run", to invoke simulators to perform the sampling as subprocesses and a method: "exit" to check the status of all jobs
* last_line: the stopping number of lines in the input file.This includes all the objective function definitions, and commands to be invoked. Optimizer will continue from here to read the rest of input. 

In [3]:
ref_dict, predict_dict, argument_dict, LAMMPS, last_line = (objective
                                                            .setup_objective
                                                            .setup(
                                                                INPUT,
                                                                TOTAL_CORES,
                                                                JOBID,
                                                                overwrite=True,
                                                                Ref_folder=Ref,
                                                                prep_folder=prep)
                                                            .finish())

# display the data structure:
print ("referenced data dictionary:\n",ref_dict)

print ("predict data dictionary:\n",predict_dict)

print ("argument dictionary:",argument_dict)

referenced data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/force_matching_tutorial/ReferenceData/force/mW_300K_1bar_500',)}}
predict data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/main_tutorial/tutorial/Predicted/force/mW_300K_1bar_500',)}}
argument dictionary: {'force': {'mW_300K_1bar_500': ('mW_300K_1bar_500', 1.0, 2, 2, 'bf 5000 eng abs w 0.0 1.0')}}


----------------------------------------------------------------------------------
###                           Initialize objective functions                     
----------------------------------------------------------------------------------
#### Purposes: 
* initialize a Python class instances that computes all objective functions specified in the input file. This object will be passed to an optimizer object, and it will be invoked each iteration by the optimizer after new force-field parameters are generated.  

#### Return: 
* "eval_objective" is a Python object that contains a method: "optimize" that peform the sampling and return objective function values for every iteration. It takes "potential type", "force-field parameters", and "status" as arguments. Another method: "update" is optional but is used to update the currently best predicted properties.  



In [ ]:
eval_objective = (objective
                  .compute_objective
                  .prepare(
                    ref_dict,
                    predict_dict,
                    argument_dict,
                    LAMMPS))

----------------------------------------------------------------------------------
###                           start optimization                                 
----------------------------------------------------------------------------------
#### Purposes: 
* Perform optimization 

#### Passd: 
* A Python class instance must be defined with "optimize" attribute, and passed to the optimizer
* input file name
* Output addresses


#### Return: 
* optimize_fm: A python class instance initialized by parsing the input file arguments
* optimization starts by calling the method "run_optimization()"

In [ ]:


optimize_fm = (optimizer
               .gradient_free
               .NelderMeadSimplex(
                   INPUT,
                   eval_objective,
                   skipped=last_line,
                   Output=JOBID+"/Output"))

# run optimization ...
optimize_fm.run_optimization()
 
